In [1]:
# env setup
!pip install pandas pyarrow tqdm regex seaborn matplotlib

import os
from pathlib import Path

BASE_DIR = Path(".").resolve()
DATA_DIR = BASE_DIR / "parquet"
RESULTS_DIR = BASE_DIR / "results2"
CONTEXT_DIR = RESULTS_DIR / "contexts2"
LLM_DIR = RESULTS_DIR / "llm_results2"

# for d in [RESULTS_DIR, CONTEXT_DIR, LLM_DIR]:
#     d.mkdir(exist_ok = True)

# print("Directories ready:")
# print(BASE_DIR, DATA_DIR, RESULTS_DIR)


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from pathlib import Path
import pandas as pd
import random

RESULTS_DIR = Path("results2")
CONTEXT_DIR = RESULTS_DIR / "contexts2"

failed_prs = pd.read_csv(RESULTS_DIR / "failed_prs_refined.csv")
failed_prs["id"] = failed_prs["id"].astype(int)

# Which ids actually have a context file?
available_ids = []
for p in CONTEXT_DIR.glob("PR*.txt"):
    try:
        pid = int(p.stem.split("_")[1])
        available_ids.append(pid)
    except Exception:
        continue

available_ids = set(available_ids)
failed_ids = set(failed_prs["id"])
candidate_ids = sorted(failed_ids & available_ids)

print(f"Failed PRs with context files: {len(candidate_ids)}")

# sample_ids = candidate_ids[:20]
# print(len(sample_ids))
# print("Sample PR IDs:", candidate_ids)



Failed PRs with context files: 7658
Sample PR IDs: [2756921963, 2757103560, 2757124156, 2757125491, 2757179026, 2757829316, 2758105942, 2758200405, 2758379503, 2758636941, 2759847657, 2760044965, 2760115428, 2760117241, 2760266883, 2760297589, 2760305985, 2761288801, 2761312980, 2761411256, 2761413721, 2761443934, 2762054930, 2762228850, 2762297369, 2762301500, 2762310020, 2763308355, 2763430169, 2763461870, 2763610953, 2763678225, 2763926843, 2763955999, 2764665933, 2765165425, 2765532800, 2765766260, 2765771643, 2765835144, 2765877627, 2766353261, 2766783930, 2766896431, 2767408688, 2767657628, 2767735199, 2767796873, 2767838200, 2767878178, 2767950239, 2767960820, 2768448959, 2768454422, 2768570229, 2768602796, 2768631398, 2768801424, 2768861546, 2769398002, 2769546631, 2769791644, 2769930216, 2769964943, 2769973551, 2770095837, 2770590659, 2770730384, 2770944230, 2771182139, 2771402284, 2771428959, 2771536472, 2771538261, 2771598346, 2771684354, 2772459642, 2773663616, 2773964481, 

In [ ]:
BASE_PROMPT = r"""
You are an assistant that classifies why GitHub pull requests failed.

A "failed" pull request in this task is one that was CLOSED WITHOUT being merged, or has remained OPEN and STALE for a long time.

Your goal is to read the pull request context and decide the SINGLE most likely primary reason why it did not get merged, according to the taxonomy below.

If there is genuinely not enough information in the context to infer a reason, you MUST use the UNKNOWN label (U1_UNKNOWN).
Do not guess when the evidence is missing or extremely weak.

TAXONOMY (reason_label → meaning):

- T1_CI_OR_TEST_FAILURE
  The PR was not merged because its tests, build, or CI checks failed, and those failures were never resolved.

- T2_CODE_QUALITY_OR_CORRECTNESS
  The PR was not merged because reviewers identified code-level problems such as logic bugs, unsafe changes, missing tests, poor design, or style violations.

- F1_REQUIREMENT_MISMATCH_OR_NOT_NEEDED
  The PR was not merged because the change did not match project requirements or direction (feature not desired, out of scope, wrong approach), even if technically possible.

- P1_LOW_PRIORITY_OR_STALE
  The PR was not merged because it was not prioritized or reviewed, became stale or stuck in the backlog, and eventually was closed mainly due to inactivity or lack of attention.

- P2_AUTHOR_UNRESPONSIVE_OR_WITHDREW
  The PR was not merged because the author stopped responding to feedback, did not address required changes, or explicitly abandoned or closed the PR.

- P3_REVIEW_OR_SOCIAL_CONFLICT
  The PR was not merged because of unresolved disagreement, unclear ownership, or other social/coordination issues in the review process, not primarily technical problems.

- P4_POLICY_OR_COMPLIANCE_ISSUE
  The PR was not merged because it violated repository or process requirements (e.g., wrong branch, missing CLA, missing required metadata or templates), not because of the technical content.

- R1_DUPLICATE_OR_ALREADY_FIXED
  The PR was not merged because another PR or commit already implemented the same change or fix, making this PR redundant.

- R2_OBSOLETE_DUE_TO_PROJECT_CHANGE
  The PR was not merged because the codebase or project direction changed so much that the PR became outdated or irrelevant (e.g., major refactor, feature removed).

- S1_INVALID_OR_SPAM
  The PR was not merged because it is not a legitimate contribution (spam, nonsense, accidental or meaningless changes).

- U1_UNKNOWN
  The PR context does not provide enough information to infer a plausible reason. Use this when there is no clear signal.

TASK:

1. Read the PR context carefully.
2. Decide the SINGLE most likely primary reason_label from the taxonomy above.
3. Write a short reason_summary (1–2 sentences) explaining why you chose that label.
4. Estimate a numeric confidence between 0 and 1 for how sure you are.

IMPORTANT:
- Choose exactly ONE reason_label.
- Do NOT invent details that are not supported by the context.
- If the signals are mixed, choose the reason that best explains why the PR ultimately failed.
- If there is not enough information, use U1_UNKNOWN.

OUTPUT FORMAT (JSON ONLY):

You MUST output exactly one JSON object with this schema:

{
  "pr_id": <integer>,
  "reason_label": "<one of T1_CI_OR_TEST_FAILURE, T2_CODE_QUALITY_OR_CORRECTNESS, F1_REQUIREMENT_MISMATCH_OR_NOT_NEEDED, P1_LOW_PRIORITY_OR_STALE, P2_AUTHOR_UNRESPONSIVE_OR_WITHDREW, P3_REVIEW_OR_SOCIAL_CONFLICT, P4_POLICY_OR_COMPLIANCE_ISSUE, R1_DUPLICATE_OR_ALREADY_FIXED, R2_OBSOLETE_DUE_TO_PROJECT_CHANGE, S1_INVALID_OR_SPAM, U1_UNKNOWN>",
  "reason_summary": "<1–2 sentence natural-language explanation>",
  "confidence": <float between 0 and 1>
}

All string values (reason_label and reason summary) MUST be enclosed in double quotes to form valid JSON.

- pr_id MUST match the pr_id given below.
- Do NOT include any extra keys.
- Do NOT include any text before or after the JSON.

Now read the PR context below and produce the JSON output.

PR_ID: {pr_id}

PR CONTEXT:
<<<
{pr_context}
>>>
"""

def build_prompt(pr_id: int, pr_context: str) -> str:
    """
    insert the PR id and context into the base prompt.
    """
    s = BASE_PROMPT.replace("{pr_id}", str(pr_id))
    s = s.replace("{pr_context}", pr_context)
    return s


In [4]:
!pip install --quiet openai
from pathlib import Path
import pandas as pd
import random
import json
import re

from openai import OpenAI



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
client = OpenAI(
    base_url="http://127.0.0.1:1234/v1",
    api_key="lm-studio"
)

LMSTUDIO_MODEL_NAME = "llama-3.2-3b-instruct"

def run_llm(prompt: str) -> str:
    """
    call the LM Studio model with the given prompt.
    returns the raw text output from the model.
    """
    response = client.chat.completions.create(
        model = LMSTUDIO_MODEL_NAME,
        messages = [
            {"role": "user", "content": prompt}
        ],
        temperature = 0.0,
        max_tokens = 512,
    )

    if not hasattr(response, "choices") or response.choices is None or len(response.choices) == 0:
        print("unexpected response from LM studio: ")
        print(response)
        raise RuntimeError("LM Studio returned no choices.")
    
    return response.choices[0].message.content

import time

def run_llm_with_retry(prompt: str, max_retries: int = 3, sleep_seconds: int = 5) -> str:
    """
    call LM Studio via run_llm(prompt), with retries on failure.
    """
    last_err = None
    for attempt in range(1, max_retries + 1):
        try:
            return run_llm(prompt)
        except Exception as e:
            last_err = e
            print(f"LLM call failed on attempt {attempt}/{max_retries}: {e}")
            if attempt < max_retries:
                print(f" Sleeping {sleep_seconds} seconds before retry...")
                time.sleep(sleep_seconds)

    raise RuntimeError(f"LLM failed after {max_retries} attempts") from last_err


In [ ]:
import json
import re

def extract_json_obj(text: str) -> dict:
    """
    extract the JSON-like block from the model output,
    add a missing closing brace if needed, and parse it.
    """

    match = re.search(r"{.*", text, re.DOTALL)
    if not match:
        raise ValueError("No JSON-like object starting with '{' found in model output.")

    json_str = match.group(0).strip()

    # if it doesn't end with '}', append one
    if not json_str.endswith("}"):
        json_str = json_str + "\n}"

    # parse as JSON
    try:
        return json.loads(json_str)
    except json.JSONDecodeError as e:
        print("⚠️Failed to json.loads the string, here is what I tried to parse:\n")
        print(json_str)
        raise e


In [9]:
from tqdm import tqdm

OUTPUT_PATH = RESULTS_DIR / "failed_pr_labels_all.csv"

# 1) Load existing results if any
if OUTPUT_PATH.exists():
    existing = pd.read_csv(OUTPUT_PATH)
    if "pr_id" in existing.columns:
        done_ids = set(existing["pr_id"].astype(int))
    else:
        done_ids = set()
    print(f"Resuming from existing results: {len(done_ids)} PRs already labeled.")
else:
    existing = None
    done_ids = set()
    print("No existing results found. Starting from scratch.")

# 2) Decide which PRs to process
remaining_ids = [pid for pid in candidate_ids if pid not in done_ids]
print(f"Remaining PRs to process: {len(remaining_ids)}")

# 3) Track if the output file existed before this run
file_existed_initially = OUTPUT_PATH.exists()
wrote_any_this_run = False

errors = []  # to keep track of PRs that failed even after retries

for pr_id in tqdm(remaining_ids):
    path = CONTEXT_DIR / f"PR_{pr_id}.txt"
    if not path.exists():
        print(f"⚠️Missing context file for PR {pr_id}, skipping.")
        continue

    try:
        with open(path, "r", encoding="utf-8") as f:
            pr_context = f.read()

        prompt = build_prompt(pr_id, pr_context)

        raw_output = run_llm_with_retry(prompt)

        obj = extract_json_obj(raw_output)

        obj["pr_id"] = int(obj.get("pr_id", pr_id))
        try:
            obj["confidence"] = float(obj.get("confidence", 0.0))
        except Exception:
            obj["confidence"] = 0.0

        row_df = pd.DataFrame([obj])

        write_header = (not file_existed_initially) and (not wrote_any_this_run)

        row_df.to_csv(OUTPUT_PATH, mode = "a", index = False, header = write_header)
        wrote_any_this_run = True

    except Exception as e:
        print(f"Error processing PR {pr_id}: {e}")
        errors.append({"pr_id": pr_id, "error": str(e)})

print("Batch run finished.")
print(f"Total errors: {len(errors)}")

if errors:
    errors_df = pd.DataFrame(errors)
    err_path = RESULTS_DIR / "failed_pr_label_errors.csv"
    errors_df.to_csv(err_path, index=False)
    print("Saved error log to:", err_path)

Resuming from existing results: 6 PRs already labeled.
Remaining PRs to process: 7652


 21%|██        | 1590/7652 [8:14:04<28:36:08, 16.99s/it]

⚠️Failed to json.loads the string, here is what I tried to parse:

{
  "pr_id": 3070903485,
  "reason_label": "T2_CODE_QUALITY_OR_CORRECTNESS",
  "reason_summary": "The PR was closed without being merged because reviewers identified code-level problems, specifically a non-conforming title that did not follow the '[\<Project\>] Title' format.",
  "confidence": 0.8
}
❌Error processing PR 3070903485: Invalid \escape: line 4 column 173 (char 249)


 34%|███▍      | 2622/7652 [13:04:35<22:54:50, 16.40s/it]

⚠️LLM call failed on attempt 1/3: Error code: 400 - {'error': 'Trying to keep the first 4152 tokens when context the overflows. However, the model is loaded with context length of only 4096 tokens, which is not enough. Try to load the model with a larger context length, or provide a shorter input'}
   Sleeping 5 seconds before retry...
⚠️LLM call failed on attempt 2/3: Error code: 400 - {'error': 'Trying to keep the first 4152 tokens when context the overflows. However, the model is loaded with context length of only 4096 tokens, which is not enough. Try to load the model with a larger context length, or provide a shorter input'}
   Sleeping 5 seconds before retry...


...

⚠️LLM call failed on attempt 3/3: Error code: 400 - {'error': 'Trying to keep the first 4128 tokens when context the overflows. However, the model is loaded with context length of only 4096 tokens, which is not enough. Try to load the model with a larger context length, or provide a shorter input'}
❌Error processing PR 3227146331: LLM failed after 3 attempts


 98%|█████████▊| 7481/7652 [35:32:59<50:32, 17.73s/it]  

⚠️Failed to json.loads the string, here is what I tried to parse:

{
  "pr_id": 3266530341,
  "reason_label": "T2_CODE_QUALITY_OR_CORRECTNESS",
  "reason_summary": "The PR was not merged because reviewers identified code-level problems such as logic bugs, unsafe changes, missing tests, poor design, or style violations (e.g., instances of `.GetProperty("`, which were not resolved by the author.",
  "confidence": 0.8
}
❌Error processing PR 3266530341: Expecting ',' delimiter: line 4 column 212 (char 288)


100%|██████████| 7652/7652 [36:19:44<00:00, 17.09s/it]

✅ Batch run finished.
Total errors: 11
Saved error log to: results2\failed_pr_label_errors.csv
